In [1]:
import gurobipy as gp
from gurobipy import GRB

def optimize_stabilizer(n, pauli_XZ, coeffs):
    """
    n: number of qubits
    pauli_XZ: list of tuples (x_vec,z_vec) in {0,1}^n
    coeffs: list of real c_t
    """
    m = len(coeffs)
    model = gp.Model("best_stabilizer_state")
    M = n

    # 1) Tableau vars
    x = model.addVars(n, n, vtype=GRB.BINARY, name="x")
    z = model.addVars(n, n, vtype=GRB.BINARY, name="z")
    r = model.addVars(n, vtype=GRB.BINARY,    name="r")

    # 2) Commutation slack
    u = model.addVars(n, n, vtype=GRB.INTEGER, name="u")

    # 3) Membership & expectation vars
    vsel   = model.addVars(m, n, vtype=GRB.BINARY,  name="vsel")
    delta  = model.addVars(m, n, vtype=GRB.INTEGER, name="delta")
    eps    = model.addVars(m, n, vtype=GRB.INTEGER, name="eps")
    y      = model.addVars(m,    vtype=GRB.BINARY,  name="y")
    sigma  = model.addVars(m,    vtype=GRB.INTEGER, name="sigma")
    zeta   = model.addVars(m,    vtype=GRB.BINARY,  name="zeta")
    p_var  = model.addVars(m,    vtype=GRB.BINARY,  name="p")
    q_var  = model.addVars(m,    vtype=GRB.BINARY,  name="q")

    # --- Commutation constraints ---
    for i in range(n):
        for k in range(i+1, n):
            model.addConstr(
               gp.quicksum(x[i,j]*z[k,j] + z[i,j]*x[k,j] for j in range(n))
               - 2*u[i,k] == 0
            )

    # --- Membership and sign constraints ---
    for t, ((x_t, z_t), c_t) in enumerate(zip(pauli_XZ, coeffs)):
        # parity‐match on X and Z (big M trick)
        for j in range(n):
            expr_x = gp.quicksum(vsel[t,i]*x[i,j] for i in range(n)) - 2*delta[t,j] - x_t[j]
            expr_z = gp.quicksum(vsel[t,i]*z[i,j] for i in range(n)) - 2*eps[t,j]   - z_t[j]
            model.addConstr( expr_x <=  M*(1-y[t]) )
            model.addConstr(-expr_x <=  M*(1-y[t]) )
            model.addConstr( expr_z <=  M*(1-y[t]) )
            model.addConstr(-expr_z <=  M*(1-y[t]) )

        # phase parity
        model.addConstr(
            gp.quicksum(vsel[t,i]*r[i] for i in range(n))
            - 2*sigma[t] - zeta[t] == 0
        )

        # link p,q to y and zeta
        model.addConstr(p_var[t] + q_var[t] == y[t])
        model.addConstr(p_var[t] <= 1 - zeta[t])
        model.addConstr(q_var[t] <=     zeta[t])

    # --- Objective ---
    obj = gp.quicksum(coeffs[t]*(p_var[t] - q_var[t]) for t in range(m))
    model.setObjective(obj, GRB.MINIMIZE)

    # --- Optimize ---
    model.Params.OutputFlag = 1
    model.optimize()

    # retrieve tableau
    X_sol = [[int(x[i,j].X) for j in range(n)] for i in range(n)]
    Z_sol = [[int(z[i,j].X) for j in range(n)] for i in range(n)]
    r_sol = [int(r[i].X) for i in range(n)]
    return X_sol, Z_sol, r_sol, model.ObjVal




In [2]:
# --- Example usage ---
n = 3
pauli_XZ = [ ([1,0,1], [0,1,1]),  ([1,1,0], [0,1,1]) ]
coeffs   = [ 0.5, -1.2 ]
X_tab, Z_tab, phases, optval = optimize_stabilizer(n, pauli_XZ, coeffs)

Restricted license - for non-production use only - expires 2026-11-23
Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Debian GNU/Linux 12 (bookworm)")

CPU model: AMD Ryzen 9 PRO 5945 12-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 6 rows, 58 columns and 14 nonzeros
Model fingerprint: 0x0db3698f
Model has 29 quadratic constraints
Variable types: 0 continuous, 58 integer (35 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 3e+00]
  Objective range  [5e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
  QRHS range       [2e+00, 4e+00]
Found heuristic solution: objective 0.0000000
Presolve removed 3 rows and 9 columns
Presolve time: 0.01s
Presolved: 212 rows, 109 columns, 577 nonzeros
Variable types: 0 continuous, 109 integer (94 b

In [3]:
import numpy as np

I = np.array([[1, 0], [0, 1]])
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])

# Build the Hamiltonian
H = (
    0.5 * np.kron(X, np.kron(Z, Y)) +
    -1.2 * np.kron(X, np.kron(Y, Z))
)

# Compute eigenvalues
eigvals = np.linalg.eigvalsh(H)
ground_energy = np.min(eigvals)
print("Ground state energy:", ground_energy)


Ground state energy: -1.7000000000000002
